# Sharing Experiments

In the [first part](https://capitalone.github.io/rubicon-ml/quick-look/logging-experiments.html)
of the quick look, we learned how to log ``rubicon_ml`` experiments in the context of a
simple classification problem. We also performed a small hyperparameter search to show how ``rubicon_ml``
can be used to compare the results of multiple model fits.

Inspecting our model fit results in the same session that we trained the model is certainly useful, but
sharing experiments can help us collaborate with teammates and compare new model training results to old
experiments.

First, we'll create a ``rubicon_ml`` entry point and get the project we logged in the first part of the
quick look.

In [1]:
from rubicon_ml import Rubicon

rubicon = Rubicon(persistence="filesystem", root_dir="./rubicon-root")
project = rubicon.get_project(name="classifying penguins")
project

Let's say we want to share the results of our hyperparmeter search with another teammate so they can evaluate the results.
``rubicon_ml``'s ``publish`` function takes a list of experiments as an input and  uses ``intake`` to generate a catalog
containing the bare-minimum amount of metadata needed to retrieve an experiment, like its ID and filepath. More on ``intake``
can be found [in their docs](https://intake.readthedocs.io/en/latest/).

Hyperparameter searches can span thousands of combos, so sharing every single file ``rubicon_ml`` logs during the training
process can be a lot. That's why we use ``intake`` via our ``publish`` function to only share what needs to be shared in a
single YAML file. Then, later, users can use said YAML file to retrieve the experiments shared within it.

**Note**: Sharing experiments relys on both the sharer and the recipient having access to the same underlying data source.
In this example, we're using a local filesystem - so these experiments couldn't actually be shared with anyone other than 
people on this same physical machine. To get the most out of sharing, log your experiments to an S3 bucket that all teammates
have access to.

In [2]:
from rubicon_ml import publish

catalog = publish(
    project.experiments(tags=["parameter search"]),
    output_filepath="./penguin_catalog.yml",
)

!head -7 penguin_catalog.yml

/Users/IEA779/Desktop/Github/Github-Public/rubicon-ml/rubicon_ml/repository/base.py:57: UserWarning: /Users/IEA779/Desktop/Github/Github-Public/rubicon-ml/notebooks/quick-look/./rubicon-root/classifying-penguins/experiments/.ipynb_checkpoints/metadata.json not found. Was this file unintentionally created?
  warnings.warn(warning)


sources:
  experiment_03b024c3_b228_451e_a1f6_9fcff2a14024:
    args:
      experiment_id: 03b024c3-b228-451e-a1f6-9fcff2a14024
      project_name: classifying penguins
      urlpath: ./rubicon-root
    driver: rubicon_ml_experiment


Each catalog contains a "source" for each ``rubicon_ml`` experiment. These sources contain the minimum metadata needed
to retrieve the associated experiment - the ``experiment_id``, ``project_name`` and ``urlpath`` to the root of the 
``rubicon_ml`` repository used as an entry point. The ``rubicon_ml_experiment`` driver can be found 
[within our library](https://github.com/capitalone/rubicon-ml/blob/main/rubicon_ml/intake_rubicon/experiment.py)
and leverages the metadata in the YAML catalog to return the experiment objects associated to it.

Provided the recipient of the shared YAML catalog has read access to the filesystem represented by ``urlpath``,
they can now use ``intake`` directly to read the catalog and load in the shared ``rubicon_ml`` expierments
for their own inspection.

In [3]:
import intake

catalog = intake.open_catalog("./penguin_catalog.yml")

for source in catalog:
    catalog[source].discover()
    
shared_experiments = [catalog[source].read() for source in catalog]

print("shared experiments:")
for experiment in shared_experiments:
    print(
        f"\tid: {experiment.id}, "
        f"parameters: {[(p.name, p.value) for p in experiment.parameters()]}, "
        f"metrics: {[(m.name, m.value) for m in experiment.metrics()]}"
    )

shared experiments:
	id: 03b024c3-b228-451e-a1f6-9fcff2a14024, parameters: [('strategy', 'most_frequent'), ('n_neighbors', 20)], metrics: [('accuracy', 0.7980769230769231)]
	id: 08625083-d5ec-4303-aeb1-8fd3b695a79a, parameters: [('strategy', 'mean'), ('n_neighbors', 15)], metrics: [('accuracy', 0.7403846153846154)]
	id: 0cc6ccee-c386-49be-a17a-c147c9526bcb, parameters: [('strategy', 'median'), ('n_neighbors', 10)], metrics: [('accuracy', 0.8653846153846154)]
	id: 0cd1373b-b7a3-4b12-9a2d-8a054c1034b9, parameters: [('strategy', 'mean'), ('n_neighbors', 5)], metrics: [('accuracy', 0.7211538461538461)]
	id: 1648cd28-2ded-4dae-9768-c5e650987ee1, parameters: [('strategy', 'median'), ('n_neighbors', 20)], metrics: [('accuracy', 0.7596153846153846)]
	id: 165c8123-aee4-4830-b7e8-6cbab7ae4586, parameters: [('strategy', 'median'), ('n_neighbors', 5)], metrics: [('accuracy', 0.8365384615384616)]
	id: 1a692338-9131-4763-b7df-c5cca8df88ae, parameters: [('strategy', 'median'), ('n_neighbors', 5)], me

## Updating Existing Catalogs

Suppose we have an existing `intake catalog` and would like to update and append experiments to that the same catalog file. To do this, let's create a project with 2 random experiments Next, in order to update an exisiting catalog file with new experiments, we can utilize the `penguin_catalog` and then directly update it with our new experiments. To do this, we leverage an optional argument in the `publish` function called `base_catalog_filepath` as seen below. The result shows the new experiments added to the `penguin catalog`. We can verify this by noticing the different project name that the new experiments fall under.


In [4]:
import intake

#Create new rubicon project and generate 2 experiments

new_project = rubicon.get_or_create_project(name="update catalog example")
new_experiments = [new_project.log_experiment() for _ in range(2)]

#publish
updated_catalog = publish(
    base_catalog_filepath="./penguin_catalog.yml",
    experiments = new_experiments,
)

!head -7 penguin_catalog.yml




sources:
  experiment_03b024c3_b228_451e_a1f6_9fcff2a14024:
    args:
      experiment_id: 03b024c3-b228-451e-a1f6-9fcff2a14024
      project_name: classifying penguins
      urlpath: ./rubicon-root
    driver: rubicon_ml_experiment
